# Benchmark CUDA FFTs using cupy and Compare with Numpy CPU
---

This notebook evaluate 2D FFT runtime for:
- Numpy on a CPU
- cupy FFT run serially
- cupy FFT run in batch mode.

In [1]:
import cupy as cp 
import time 
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def measure_fft_timeNP(shape, num_trials=10): 
    data = np.random.random(shape) + 1j * np.random.random(shape)
    #cp.cuda.Device(0).synchronize() 
    start_time = time.time() 
    for _ in range(num_trials): 
        np.fft.fft2(data) 
        #cp.cuda.Device(0).synchronize() 
        return (time.time() - start_time) / num_trials 

In [3]:
def measure_fft_time(shape, num_trials=10): 
    data = cp.random.random(shape) + 1j * cp.random.random(shape)
    cp.cuda.Device(0).synchronize() 
    start_time = time.time() 
    for _ in range(num_trials): 
        cp.fft.fft2(data) 
        cp.cuda.Device(0).synchronize() 
    return (time.time() - start_time) / num_trials 

In [4]:
def batch_fft_2d(n, m, batch_size):
    """
    Perform batch 2D FFTs on the GPU using CuPy, and measure execution time.

    Parameters:
    - n: number of rows in each FFT
    - m: number of columns in each FFT
    - batch_size: the number of 2D FFTs to process in parallel

    Returns:
    - output_data: The result of the batch of FFTs.
    - execution_time: Time taken to execute the FFTs in milliseconds.
    """
    # Create a batch of random input data (complex)
    input_data = cp.random.randn(batch_size, n, m).astype(cp.complex64)

    # Create CUDA events to measure time
    start_event = cp.cuda.Event()
    end_event = cp.cuda.Event()

    # Record the start time
    start_event.record()

    # Execute the batch of 2D FFTs on the GPU using CuPy's FFT
    output_data = cp.fft.fft2(input_data)

    # Record the end time
    end_event.record()

    # Wait for the event to complete
    end_event.synchronize()

    # Calculate the elapsed time in milliseconds
    execution_time = cp.cuda.get_elapsed_time(start_event, end_event)  # Time in milliseconds

    # Transfer the result back to the CPU
    output_data_cpu = cp.asnumpy(start_event, end_event)  # CuPy array -> NumPy array

    return output_data_cpu, execution_time/batch_size/1000.


In [5]:
def prime_factors(n):
    factors = []

    while n % 2 == 0:
        factors.append(2)
        n //= 2
    for i in range(3, int(n**0.5) + 1, 2):
        while n % i == 0:
            factors.append(i)
            n //= i

    if n > 2:
        factors.append(n)
    return factors

## CPU Numpy FFT

In [ ]:
fftMinNP, fftMaxNP = 64, 512
x = np.linspace(fftMinNP, fftMaxNP, num=fftMaxNP-fftMinNP+1)
y = np.linspace(fftMinNP, fftMaxNP, num=fftMaxNP-fftMinNP+1)
X, Y = np.meshgrid(x, y)
resultNP = np.zeros(X.shape)
i = 0
for i in range(fftMinNP, fftMaxNP):
    if(i % 10 == 0):
        print(f'{i}-', end='')
    for j in range(fftMinNP, fftMaxNP): 
        resultNP[i - fftMinNP, j-fftMinNP] = measure_fft_timeNP((i,j), num_trials=1)     

70-80-90-100-110-120-130-140-150-160-170-180-190-200-

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
im = ax.imshow(resultNP, interpolation='None',  vmin=0, vmax=0.008, extent=[fftMinNP, fftMaxNP, fftMinNP, fftMaxNP])
cb = fig.colorbar(im)
cb.set_label('fft time (s)')

In [ ]:
plt.plot(resultNP[192-64, :], '.')

Compute multiple FFTs in parallel.

## CUDA sequential FFTs

This calculates each FFT as a sequential request to the GPU.

In [ ]:
fftMin, fftMax = 64, 512
num_trials = 25
x = np.linspace(fftMin, fftMax, num=fftMax-fftMin+1)
y = np.linspace(fftMin, fftMax, num=fftMax-fftMin+1)
X, Y = np.meshgrid(x, y)
resultSingle = np.zeros(X.shape)
maxF = np.zeros(X.shape)
for i in range(fftMin, fftMax+1):
    if(i % 10 == 0):
        print(f'{i}-', end='')
    for j in range(fftMin, fftMax+1): 
        resultSingle[i - fftMin, j - fftMax] = measure_fft_time((i,j), num_trials=num_trials) 
        maxF[i - fftMin, j - fftMax] = max(prime_factors(i) + prime_factors(j))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(18, 12))
im = ax[0].imshow(resultSingle, vmax=.0005, interpolation='None', extent=[fftMin, fftMax, fftMin, fftMax], origin='lower')
cb = fig.colorbar(im)
cb.set_label('fft time (s)')
im1 = ax[1].imshow(maxF, interpolation='None', extent=[fftMin, fftMax, fftMin, fftMax], origin='lower')
cb = fig.colorbar(im1)

In [ ]:
x = resultSingle[250, :]
plt.plot(range(fftMin, fftMax+1), x, '.')

In [ ]:
for i in [(192, 98), (224, 192), (212, 144), (216, 144), (212+8, 144+8), (224, 160),   (96, 64)]:
    fft_time = measure_fft_time(i, num_trials=200)
    rgLargest = max(prime_factors(i[0]))
    azLargest = max(prime_factors(i[1]))        
    print(f'rg={i[0]}, az={i[1]} fft={fft_time:.8f}  largest primes (rg, az)= {rgLargest}, {azLargest}')
for i in [(192, 98), (224, 192), (212, 144), (216, 144), (212+8, 144+8), (224, 160),   (96, 64)]:
    fft_time = measure_fft_time(i, num_trials=200)
    rgLargest = max(prime_factors(i[0]))
    azLargest = max(prime_factors(i[1]))        
    print(f'rg={i[0]}, az={i[1]} fft={fft_time:.8f}  largest primes (rg, az)= {rgLargest}, {azLargest}')

## Batch FFT

Compute multiple FFTs in parallel.

In [ ]:
resultBatch = np.zeros(X.shape)
for i in range(fftMin, fftMax+1):
    if(i % 10 == 0):
        print(f'{i}-', end='')
    for j in range(fftMin, fftMax+1): 
        resultBatch[i - fftMin, j - fftMin] = batch_fft_2d(i, j, num_trials)[1] 

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(18, 12))
im = ax[0].imshow(resultsBatch, vmax=.0005, interpolation='None', extent=[fftMin, fftMax, fftMin, fftMax], origin='lower')
cb = fig.colorbar(im)
cb.set_label('fft time (s)')
im1 = ax[1].imshow(maxF, interpolation='None', extent=[fftMin, fftMax, fftMin, fftMax], origin='lower')
cb = fig.colorbar(im1)

In [ ]:
resultBatch.shape

In [ ]:
x = resultBatch[192-64, :]
plt.plot(x)
print(np.argmax(x))
print(prime_factors(np.argmax(x)))